### Features för analys 
Skapa nya kolumner för analys (t.ex. veckodag, månad, försäljning per vecka, ledtider)

- Utgår fran passadatum - Därför att de beskriver när aktivitet faktiskt sker.

**Tid (när passet sker)**
- **veckodag**: vilken dag, t.ex. måndag.
- **vecka**: vilken vecka på året.
- **månad**: vilken månad.
- **år**: vilket år.
- **helg**: är det lördag/söndag? (ja/nej)
- **tid_på_dag**: natt, morgon, dag eller kväll.

**Bokning**
- **ledtid_dagar**: hur många dagar innan passet man bokade.

**Efterfrågan (hur populärt det är)**
- **pass_popularitet_vecka**: hur många som bokat samma pass den veckan.
- **anläggning_belastning_vecka**: hur många bokningar gymmet haft den veckan.

**Ny medlemstyp (utifrån pris)**
- **medlemstyp_ny**: gruppen baserat på vad man betalar.
  - 0 kr = **VIP**
  - 249 kr = **Student**
  - 349 kr = **Student+** eller **Bas** (beroende på vad personen var innan)
  - 449 kr = **Student+** eller **Bas+**
  - 599 kr = **Premium**

In [1]:
# Importera bibliotek
import sqlite3
import pandas as pd
import numpy as np

In [5]:
# Läs all data från SQLite
conn = sqlite3.connect('friskvard_data_cleaned.db')
df_all_cleaned = pd.read_sql('SELECT * FROM friskvard_data', conn)
conn.close()

df_clean = df_all_cleaned[df_all_cleaned['dataset'] == 'main'].copy()
df_val = df_all_cleaned[df_all_cleaned['dataset'] == 'validation'].copy()

# Skapar en kopia för feature engineering
df_feat = df_all_cleaned.copy()

In [6]:
df_feat.head()

,bokning_id,medlem_id,medlemstyp,medlem_startdatum,medlem_slutdatum,månadskostnad,födelseår,pass_id,passnamn,anläggning,...,bokningsdatum,passdatum,passtid,status,feedback_text,feedbackdatum,feedback_betyg,är_negativt_belopp,månadskostnad_abs,dataset
0,BOK-000001,MED-10158,Premium,2023-07-29 00:00:00,2025-03-07 00:00:00,599,2006.0,PASS-2024-10-01-001,Yoga,Göteborg Centrum,...,2024-09-24 00:00:00,2024-01-10 00:00:00,11:00:00.000000,Genomförd,Bästa yogapasset jag varit på. Kommer tillbaka!,2024-02-10 00:00:00,4.0,0,599,main
1,BOK-000002,MED-10229,Premium,2023-01-08 00:00:00,2025-06-26 00:00:00,599,1984.0,PASS-2024-12-04-002,Spinning,Malmö Centrum,...,2024-11-27 00:00:00,2024-04-12 00:00:00,10:00:00.000000,Genomförd,Professionellt och välorganiserat pass.,2024-05-12 00:00:00,5.0,0,599,main
2,BOK-000003,MED-10223,Student,2023-12-27 00:00:00,2025-12-05 00:00:00,249,2006.0,PASS-2024-11-09-003,Styrketräning,Okänd,...,2024-08-11 00:00:00,2024-09-11 00:00:00,18:00:00.000000,Genomförd,,NaN,NaN,0,249,main
3,BOK-000004,MED-10110,Bas,2023-07-03 00:00:00,2025-03-24 00:00:00,349,2006.0,PASS-2024-09-04-004,Pilates,Göteborg Centrum,...,2024-01-09 00:00:00,2024-09-04 00:00:00,07:00:00.000000,Genomförd,Professionellt och välorganiserat pass.,2024-02-09 00:00:00,5.0,0,349,main
4,BOK-000005,MED-10022,Premium,2022-05-11 00:00:00,2025-01-06 00:00:00,599,1992.0,PASS-2024-10-29-005,Pilates,Västerås,...,2024-10-26 00:00:00,2024-10-29 00:00:00,20:30:00.000000,Genomförd,Bästa pilatespasset jag varit på. Kommer tillb...,2024-10-30 00:00:00,5.0,0,599,main


In [ ]:
# Kontrollera antal rader och unika bokning_id
print('Antal rader (alla):', len(df_all_cleaned))
print("Unika bokning_id (alla):", df_all_cleaned['bokning_id'].nunique())

# Unikhet per dataset (bokning_id återanvänds mellan main/validation)
if 'dataset' in df_all_cleaned.columns:
    print('\nUnika bokning_id per dataset:')
    print(df_all_cleaned.groupby('dataset')['bokning_id'].nunique())

# Unikhet för kombinerad nyckel
df_all_cleaned['bokning_id_key'] = (
    df_all_cleaned['dataset'].astype(str) + '_' + df_all_cleaned['bokning_id'].astype(str)
    if 'dataset' in df_all_cleaned.columns
    else df_all_cleaned['bokning_id'].astype(str)
 )
print("\nUnika bokning_id_key (alla):", df_all_cleaned['bokning_id_key'].nunique())

Antal rader: 3450
Antal unika rader 'bokning_id': 3000


In [4]:
# Skapa ett feature-set från rengjord dataframe
df_feat = df_all_cleaned.copy()

# === Tidsegenskaper för passdatum ===
df_feat['veckodag'] = df_feat['passdatum'].dt.day_name()
df_feat['vecka'] = df_feat['passdatum'].dt.isocalendar().week.astype('Int64')
df_feat['månad'] = df_feat['passdatum'].dt.month.astype('Int64')
df_feat['år'] = df_feat['passdatum'].dt.year.astype('Int64')
df_feat['helg'] = df_feat['passdatum'].dt.dayofweek.isin([5, 6])

# === Tid på dagen (från passtid) ===
passtid_dt = pd.to_datetime(df_feat['passtid'].astype(str), errors='coerce')
df_feat['tid_på_dag'] = pd.cut(
    passtid_dt.dt.hour,
    bins=[-1, 5, 11, 17, 23],
    labels=['Natt', 'Morgon', 'Dag', 'Kväll']
    ).astype('category')

# === Ledtid (bokningsdatum -> passdatum) ===
df_feat['ledtid_dagar'] = (df_feat['passdatum'] - df_feat['bokningsdatum']).dt.days.astype('Int64')

# === Popularitet och belastning ===
df_feat['pass_popularitet_vecka'] = (
    df_feat.groupby(['passnamn', 'år', 'vecka'])['bokning_id']
    .transform('count')
    )

df_feat['anläggning_belastning_vecka'] = (
    df_feat.groupby(['anläggning', 'år', 'vecka'])['bokning_id']
    .transform('count')
    )

# === Ny medlemsindelning med bins + labels ===
kostnad = df_feat['månadskostnad_abs']
medlemstyp_norm = df_feat['medlemstyp'].astype(str).str.lower()

# Grundindelning efter pris
df_feat['medlemstyp_ny'] = pd.cut(
    kostnad,
    bins=[-1, 0, 249, 349, 449, 599, float('inf')],
    labels=['VIP', 'Student', '349', '449', 'Premium', 'Övrigt'],
    include_lowest=True
    ).astype('object')

# Förfina 349/449 baserat på befintlig medlemstyp
df_feat['medlemstyp_ny'] = df_feat['medlemstyp_ny'].replace({'349': 'Bas', '449': 'Bas+'})
df_feat['medlemstyp_ny'] = np.where(
    (df_feat['medlemstyp_ny'] == 'Bas') & medlemstyp_norm.str.contains('student', na=False),
    'Student+',
    df_feat['medlemstyp_ny']
    )
df_feat['medlemstyp_ny'] = np.where(
    (df_feat['medlemstyp_ny'] == 'Bas+') & medlemstyp_norm.str.contains('student', na=False),
    'Student+',
    df_feat['medlemstyp_ny']
    )

df_feat['medlemstyp_ny'] = df_feat['medlemstyp_ny'].astype('category')
df_feat['medlemstyp_ny'].value_counts(dropna=False)

# === Rekrytering per månad och medlemstyp_ny ===
rekrytering_per_månad = (
    df_feat.dropna(subset=['medlem_startdatum'])
    .assign(start_månad=df_feat['medlem_startdatum'].dt.to_period('M'))
    .groupby(['start_månad', 'medlemstyp_ny'])['medlem_id']
    .nunique()
    .rename('rekryteringar')
    .reset_index()
 )

# Visa
rekrytering_per_månad.sort_values(['start_månad', 'medlemstyp_ny']).head(12)
df_feat.head(10)

AttributeError: Can only use .dt accessor with datetimelike values

### Sentimentalanalys
Klassificera recensioner/feedback som positiv, neutral eller negativ med BERT eller LLMlanalys

In [ ]:
# device=0 använder GPU, device=-1 använder CPU
# Importera bibliotek
from transformers import pipeline

# Ladda sentimentmodellen
classifier = pipeline(
    "sentiment-analysis",
    model="KBLab/robust-swedish-sentiment-multiclass",
    device=0 if torch.cuda.is_available() else -1
)

In [ ]:
# === Enkel sentimentanalys ===

# Små listor med positiva/negativa svenska ord
pos_ord = {
    'bra','bäst','fantastisk','toppen','nöjd','nöjda','älskar','älskade','trevlig','trevliga',
    'snäll','snälla','hjälpsam','hjälpsamma','proffsig','proffsiga','ren','fräsch','fräscht',
    'kul','roligt','rekommenderar','gillar','glad','super','klockren','perfekt'
}
neg_ord = {
    'dålig','dåligt','sämst','missnöjd','tråkig','tråkigt','besviken','dyr','smutsig','stökig',
    'otrevlig','otrevligt','kall','varm','fel','problem','jobbigt','kaos','aldrig','inte','försenad'
}

def enkel_sentiment(text):
    if pd.isna(text) or not str(text).strip():
        return 'neutral'
    text = str(text).lower()
    pos = sum(ord in text for ord in pos_ord)
    neg = sum(ord in text for ord in neg_ord)
    if pos > neg:
        return 'positiv'
    if neg > pos:
        return 'negativ'
    return 'neutral'

df_feat['sentiment_enkel'] = df_feat['feedback_text'].apply(enkel_sentiment).astype('category')

df_feat['sentiment_enkel'].value_counts(dropna=False)